In [1]:
import helpy
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [2]:
im_file='/home/zhejun/Result/20181203/result_00001.tif'

In [4]:
data =helpy.load_data('/home/zhejun/Result/20181203/result')

In [5]:
data['o'] = (data['o'] + np.pi)%(2*np.pi)

In [6]:
frame1 = data[data['f']==0]

In [7]:
rescale_size = 4

In [8]:
training_shape = 32
centralize  = training_shape//2

In [9]:
pos = frame1['xy'] - np.array([centralize,centralize])

In [10]:
pos_start = pos.astype(int)

In [11]:
pos_cen = (frame1['xy'] - pos_start)* rescale_size

In [12]:
pos_arrow = pos_cen + rescale_size * 15.0 * np.array([np.cos(frame1['o']), np.sin(frame1['o'])]).T

In [13]:
pos_arrow = pos_arrow.astype(np.float32)

In [14]:
pos_cen = pos_cen.astype(np.float32)

In [15]:
def prep_image(im_file, width = 2):
    im = plt.imread(im_file).astype(float)
    s = width*im.std()
    m = im.mean()
    im -= m - s
    im /= 2 * s
    np.clip(im,0,1,out=im)
    return im

In [46]:
img = prep_image(im_file)

In [47]:
origin_img = np.copy(img)

In [48]:
number_test = 5

In [49]:
from skimage.transform import resize

In [57]:
test_list= list()
test_list.insert(0,[(3,5)])

In [58]:
test_list

[[(3, 5)]]

In [71]:
def orientation_click(event, x, y, flags, param):
    global button_down
    
    if event == cv2.EVENT_LBUTTONUP and button_down:
        button_down = False
        points_store.append((x,y))
        #cv2.circle(resize_img, points_store[0] ,(x, y), 2, (255,0,0))
        cv2.arrowedLine(resize_img, points_store[0], points_store[1], (255,0,0),1)
        cv2.imshow("test", resize_img)
        #print points_store
        
    elif event == cv2.EVENT_MOUSEMOVE and button_down:
        image = resize_img.copy()
        cv2.arrowedLine(image, points_store[0], (x,y), (255,0,0),1)
        cv2.imshow("test", image)
    
    elif event == cv2.EVENT_LBUTTONDOWN and len(points_store) < 2:
        button_down = True
        points_store.insert(0, (x,y))
        


In [80]:
np.arctan2(1,np.sqrt(3))

0.5235987755982989

In [89]:
np.rad2deg(np.arctan2(1,np.sqrt(3)))

30.000000000000004

In [1]:
training_x=list()
training_regress= list()
training_class = list()
test_x = list()
cv2.namedWindow("test",cv2.WINDOW_NORMAL)
cv2.resizeWindow("test",400,400)
pi2 = 2*np.pi
for frame, (x_start,y_start),arr_cen, arr_pos in zip(frame1['o'][:number_test],pos_start[:number_test],pos_cen[:number_test],pos_arrow[:number_test]):
    temp_img = img[x_start:x_start+training_shape, y_start:y_start + training_shape]
    resize_img = resize(temp_img, (training_shape*rescale_size, training_shape*rescale_size))
    if np.isnan(arr_pos[0]):
        cv2.imshow("test", resize_img)
    else:
        arrow_plot = cv2.arrowedLine(resize_img, (arr_cen[1],arr_cen[0]),(arr_pos[1], arr_pos[0]),(255,0,0),2)
        cv2.imshow('test', resize_img)
    print "detect or is:", frame
    press = cv2.waitKey(0)
    # everything is good, store current information into training sets
    if press == 49:
        training_x.append(temp_img)
        training_regress.append(frame)
        training_class.append(1)
    # enter plot mode
    elif press == 50: 
        points_store = []
        button_down = False
        cv2.imshow("test", resize_img)
        cv2.setMouseCallback('test',orientation_click, points_store)
        enter = cv2.waitKey(0)
        orientation = np.arctan2(points_store[1][0]-points_store[0][0], points_store[1][1]-points_store[0][1])
        orientation = (orientation + pi2)%pi2
        if enter == 10:
            training_x.append(temp_img)
            training_regress.append(orientation)
            training_class.append(1)
    elif press == 51:
        training_x.append(temp_img)
        training_regress.append(np.nan)
        training_class.append(0)
    elif press == 27:
        break
cv2.destroyAllWindows()
for i in range (1,5):
    cv2.waitKey(1)

NameError: name 'cv2' is not defined

In [92]:
training_regress

[3.837428, 2.3602843, 2.324290425707323, 2.3670636273696157, 0.644967383402296]

## 

In [ ]:
def draw_circle(event,x,y,flags,param):
    if event == cv2.EVENT_LBUTTONDOWN:
        #cv2.arrowedLine(resize_img,(arr_cen[1],arr_cen[0]), (x,y),(255,255,255),2)
        param = x, y
        cv2.circle(resize_img, (x1,y1),2,(255,0,0),-1)
        cv2.imshow("test", resize_img)
        waitkey = cv2.waitKey(0)
        if waitkey == 10:
            print 1
        elif waitkey == 27:
            
    #if event == cv2.EVENT_LBUTTONDOWN:
        #x2, y2 = x,y
        #cv2.arrowedLine(resize_img,(x1,y1),(x2,y2),(255,0,0),2)
        #y_axis = arr_cen[1] - x
        #x_axis = arr_cen[0] - y
    #orientation = np.arctan2(y1-y2, x1-x2) + np.pi
    #print(orientation)
    #waitkey = cv2.waitKey(0)
    #if waitkey==10:
        #training_x.append(temp_img)
        #training_regress.append(orientation)
        #training_class.append(1)

In [ ]:
# training_x=list()
training_y=list()
test_x = list()
cv2.namedWindow("test",cv2.WINDOW_NORMAL)
cv2.resizeWindow("test",1000,1000)
for frame, (x_start,y_start) in zip(frame1[:number_test],pos_start[:number_test]):
    cv2.imshow('test', img[x_start:x_start+32, y_start:y_start+32])
    press = cv2.waitKey(0)
    if press==10:
        training_x.append(origin_img[x_start:x_start+32,y_start:y_start+32])
        training_y.append(frame['o'])
    elif press == 32:
        training_x.append(origin_img[x_start:x_start+32,y_start:y_start+32])
        training_y.append(-1)
    elif press == 27:
        test_x.append(origin_img[x_start:x_start+32,y_start:y_start+32])
cv2.destroyAllWindows()
for i in range (1,5):
    cv2.waitKey(1)

In [183]:
xx = np.array(training_x)

In [185]:
training_y

[3.8374281,
 2.3602843,
 2.3842254,
 2.2701907,
 2.390636,
 2.5056405,
 2.2258554,
 -1,
 3.6374664,
 0.96984053,
 2.6277766,
 2.0499349,
 3.5345984,
 2.4549937,
 2.2713456,
 2.737061,
 2.2961869,
 2.4313397,
 3.9772291,
 2.5481315,
 2.1931863,
 1.1282506,
 1.9731727,
 3.6567349,
 2.7351785,
 1.9367518,
 1.9833751,
 2.7331414,
 2.3631883,
 2.3635335,
 2.1926093,
 2.5066237,
 2.303576,
 1.9302411,
 1.8536468,
 1.3947892,
 2.4837737,
 4.2944603,
 2.7365022,
 1.9737086,
 2.9698405]

In [37]:
import cv2
import numpy as np

ix,iy = -1,-1
# mouse callback function
def draw_circle(event,x,y,flags,param):
    global ix,iy
    if event == cv2.EVENT_LBUTTONDBLCLK:
        cv2.circle(img,(x,y),100,(255,0,0),-1)
        ix,iy = x,y

# Create a black image, a window and bind the function to window
img = np.zeros((512,512,3), np.uint8)
cv2.namedWindow('image')
cv2.setMouseCallback('image',draw_circle)

while(1):
    cv2.imshow('image',img)
    k = cv2.waitKey(20) & 0xFF
    if k == 27:
        break
    elif k == ord('a'):
        print ix,iy
cv2.destroyAllWindows()
for i in range(1,5):
    cv2.waitKey(1)

291 376
291 376
291 376
291 376
291 376
291 376
291 376
291 376
291 376
291 376
235 209


In [43]:
import cv2
from time import time
boxes = []
def on_mouse(event, x, y, flags, params):
    # global img
    t = time()

    if event == cv2.EVENT_LBUTTONDOWN:
         print 'Start Mouse Position: '+str(x)+', '+str(y)
         sbox = [x, y]
         boxes.append(sbox)
         # print count
         # print sbox

    elif event == cv2.EVENT_LBUTTONUP:
        print 'End Mouse Position: '+str(x)+', '+str(y)
        ebox = [x, y]
        boxes.append(ebox)
        print boxes
        crop = img[boxes[-2][1]:boxes[-1][1],boxes[-2][0]:boxes[-1][0]]

        cv2.imshow('crop',crop)
        k =  cv2.waitKey(0)
        if ord('r')== k:
            cv2.imwrite('Crop'+str(t)+'.jpg',crop)
            print "Written to file"

count = 0
while(1):
    count += 1
    img = np.zeros((256,256))
    # img = cv2.blur(img, (3,3))
    img = cv2.resize(img, None, fx = 0.25,fy = 0.25)

    cv2.namedWindow('real image')
    cv2.setMouseCallback('real image', on_mouse, 0)
    cv2.imshow('real image', img)
    if count < 50:
        if cv2.waitKey(33) == 27:
            cv2.destroyAllWindows()
            break
    elif count >= 50:
        if cv2.waitKey(0) == 27:
            cv2.destroyAllWindows()
            break
        count = 0

Start Mouse Position: 22, 24
End Mouse Position: 22, 24
[[22, 24], [22, 24]]


error: /home/travis/miniconda/conda-bld/work/opencv-2.4.11/modules/highgui/src/window.cpp:261: error: (-215) size.width>0 && size.height>0 in function imshow


Start Mouse Position: 36, 39
End Mouse Position: 36, 39
[[22, 24], [22, 24], [36, 39], [36, 39]]


error: /home/travis/miniconda/conda-bld/work/opencv-2.4.11/modules/highgui/src/window.cpp:261: error: (-215) size.width>0 && size.height>0 in function imshow


Start Mouse Position: 20, 32
End Mouse Position: 20, 32
[[22, 24], [22, 24], [36, 39], [36, 39], [20, 32], [20, 32]]


...

Start Mouse Position: 46, 29
End Mouse Position: 47, 29
[[22, 24], [22, 24], [36, 39], [36, 39], [20, 32], [20, 32], [46, 29], [47, 29]]


error: /home/travis/miniconda/conda-bld/work/opencv-2.4.11/modules/highgui/src/window.cpp:261: error: (-215) size.width>0 && size.height>0 in function imshow


KeyboardInterrupt: 

Start Mouse Position: 28, 13
End Mouse Position: 28, 13
[[22, 24], [22, 24], [36, 39], [36, 39], [20, 32], [20, 32], [46, 29], [47, 29], [28, 13], [28, 13]]


error: /home/travis/miniconda/conda-bld/work/opencv-2.4.11/modules/highgui/src/window.cpp:261: error: (-215) size.width>0 && size.height>0 in function imshow


In [ ]:
cv2.set

## 

In [ ]:
cv2.namedWindow("test",cv2.WINDOW_NORMAL)
cv2.resizeWindow("test",1000,1000)
for start, arrow in zip(pos_cen[:number_test],pos_arrow[:number_test]):
    if np.isnan(arrow[0]):
        continue
    arrow_plot = cv2.line(img,(start[1],start[0]),(arrow[1],arrow[0]),(255,0,0),1)